In [1]:
import numpy as np 
import pandas as pd 
import os
import json
import skimage
import zipfile
from skimage.io import imread, imshow
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import CondensedNearestNeighbour
from matplotlib import pyplot
from numpy import where
import random
import matplotlib.pyplot as plt
from skimage.filters import prewitt_h,prewitt_v
from skimage.feature import hog
import gc
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from PIL import Image
import pickle
import bz2
import _pickle as cPickle
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler, Normalizer
import skimage
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier, RidgeClassifier,LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score,confusion_matrix,ConfusionMatrixDisplay
from sklearn.decomposition import PCA


In [3]:
metadata = []
for dirname, _, filenames in os.walk('/kaggle/input/amazon-bucket'):
    for filename in filenames:
        metadata.append(os.path.join(dirname, filename))

In [4]:
imagedata = []
for dirname, _, filenames in os.walk('/kaggle/input/amazonimage'):
    for filename in filenames:
        imagedata.append(os.path.join(dirname, filename))

In [5]:
with open(metadata[0], "r") as f:
    print(json.load(f))

In [6]:
image = imread(imagedata[0],as_gray = True)
imshow(image)

In [7]:
m1 = []
im1 = []
for meta in metadata:
    meta = meta.replace("/kaggle/input/amazon-bucket/METADATA/","")
    meta = meta.replace(".json","")
    m1.append(meta)

In [8]:
for img in imagedata:
    img = img.replace("/kaggle/input/amazonimage/IMAGEDATA/","")
    img = img.replace(".jpg","")
    im1.append(img)

In [9]:
m11 = set(m1)
intersect = m11.intersection(im1)
intersect = list(intersect)

In [10]:
newmetadata = []
newimagedata = []
for inter in intersect:
    newmetadata.append("/kaggle/input/amazon-bucket/METADATA/"+inter+".json")
    newimagedata.append("/kaggle/input/amazonimage/IMAGEDATA/"+inter+".jpg")

In [11]:
metafiles = []
for i,file in enumerate(newmetadata):
    with open(file, "r") as f:
        metafiles.append(json.load(f))

In [12]:
quantity = []
keepos = []
def get_total_quantity(metafiles):
    number = []
    for i,meta in enumerate(metafiles):
        if meta['EXPECTED_QUANTITY']<13:
            number.append(meta['EXPECTED_QUANTITY'])
            keepos.append(i)
    return number

In [13]:
quantity = get_total_quantity(metafiles)
print(len(quantity))

In [15]:
positions = []
for i in range(13):
    positions.append([])
for i,num in enumerate(quantity):
    positions[num].append(keepos[i])

In [17]:
newpositions = []
for i in range(len(positions)):
    if len(positions[i])>1600:
        division = int(len(positions[i])/1600)
        newpositions.append(random.sample(positions[i], int(len(positions[i])/division)))
    else:
        newpositions.append(positions[i])

In [18]:
qqq =[]
for i in range(13):
    qq = []
    for j in range(len(newpositions[i])):
        qq.append(i)
    qqq.append(qq)

In [20]:
plt.hist(qqq, width=0.85,bins=7)
plt.show()

In [21]:
finallist = []
for pos in newpositions:
    finallist = finallist+pos

In [23]:
 class RGB2GrayTransformer(BaseEstimator, TransformerMixin):
    """
    Convert an array of RGB images to grayscale
    """
 
    def __init__(self):
        pass
 
    def fit(self, X, y=None):
        """returns itself"""
        return self
 
    def transform(self, X, y=None):
        """perform the transformation and return an array"""
        return np.array([skimage.color.rgb2gray(img) for img in X])
     
class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """
 
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
 
        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)
 
        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:
            return np.array([local_hog(img) for img in X])

In [24]:
imagefiles = []
jj = 0
for i,img in enumerate(newimagedata):
    if i in finallist:
        img1 = Image.open(img).convert("RGB")
        img1 = img1.resize((160,160),Image.BILINEAR)
        #150,150
        img1.save("Image1.jpg")
        #imagefiles.append(imread("Image1.jpg",as_gray = True))
        image = imread("Image1.jpg")
        #edges_prewitt_vertical = prewitt_v(image)
        imagefiles.append(image)
        #with bz2.BZ2File("ImagesPixels"+str(i+1)+".pbz2", 'w') as f: 
        #    cPickle.dump(image, f)

In [25]:
newquantity = []
for i,meta in enumerate(metafiles):
    if i in finallist:
        newquantity.append(meta['EXPECTED_QUANTITY'])

In [28]:
del metafiles
del newimagedata
del newmetadata
del quantity

In [29]:
gc.collect()

In [30]:
image_train, image_test, quan_train, quan_test = train_test_split(imagefiles, newquantity, test_size=0.25, random_state=42,stratify=newquantity)

In [31]:
del qqq
del finallist

In [32]:
gc.collect()

In [33]:
# create an instance of each transformer
grayify = RGB2GrayTransformer()
hogify = HogTransformer(
    pixels_per_cell=(14, 14), 
    cells_per_block=(2,2), 
    orientations=9, 
    block_norm='L2-Hys'
)
scalify = StandardScaler()
 
# call fit_transform on each transform converting X_train step by step
X_train_gray = grayify.fit_transform(image_train)
X_train_hog = hogify.fit_transform(X_train_gray)
X_train_prepared = scalify.fit_transform(X_train_hog)

In [34]:
svm_lin = SVC(kernel="linear", max_iter=1000, probability=True)  
svm_lin.fit(X_train_prepared, quan_train)
predictions = svm_lin.predict(X_test_prepared)
print("The accuracy of linear model is: " + str(svm_lin.score(X_test_prepared, quan_test)))
fig,ax = plt.subplots(figsize=(10, 5))
cm = confusion_matrix(quan_test,predictions,labels=np.unique(quantity))
disp = ConfusionMatrixDisplay(cm, display_labels=np.unique(quantity))
disp.plot(ax=ax)
plt.savefig('lin.jpg')
plt.show()

In [35]:
svm_rbf = SVC(kernel="rbf", max_iter=500, probability=True)   
svm_rbf.fit(X_train_prepared, quan_train)
predictions = svm_rbf.predict(X_test_prepared)
print("Kernel = rbf: " + str(svm_rbf.score(X_test_prepared, quan_test)))
fig,ax = plt.subplots(figsize=(10, 5))
cm = confusion_matrix(quan_test,predictions, labels=np.unique(quantity))
disp = ConfusionMatrixDisplay(cm, display_labels=np.unique(quantity))
disp.plot(ax=ax)
plt.savefig('rbf.jpg')
plt.show()


In [ ]:
svm_poly = SVC(kernel="poly", max_iter=500, probability=True)   
svm_poly.fit(X_train_prepared, quan_train)
predictions = svm_poly.predict(X_test_prepared)
print("Kernel = poly: " + str(svm_poly.score(X_test_prepared, quan_test)))
fig,ax = plt.subplots(figsize=(10, 5))
cm = confusion_matrix(quan_test,predictions, labels=np.unique(quantity))
disp = ConfusionMatrixDisplay(cm, display_labels=np.unique(quantity))
disp.plot(ax=ax)
plt.savefig('poly.jpg')
plt.show()

In [37]:
#gnb = RandomForestClassifier(n_estimators=500)
gnb = SGDClassifier(random_state=42, max_iter=2000, tol=1e-3)
#gnb = SVC()
#gnb = AdaBoostClassifier()
#gnb = MLPClassifier(solver='sgd',max_iter=2000)
gnb.fit(X_train_prepared,quan_train)
X_test_gray = grayify.transform(image_test)
X_test_hog = hogify.transform(X_test_gray)
X_test_prepared = scalify.transform(X_test_hog)
predictions = gnb.predict(X_test_prepared)
print(f1_score(quan_test,predictions,average='macro'))
fig,ax = plt.subplots(figsize=(20, 10))
cm = confusion_matrix(quan_test,predictions,labels=np.unique(newquantity))
disp = ConfusionMatrixDisplay(cm, display_labels=np.unique(newquantity))
disp.plot(ax=ax)
plt.show()
print(gnb.score(X_test_prepared, quan_test))

In [38]:
gnb = AdaBoostClassifier()
gnb.fit(X_train_prepared,quan_train)
predictions = gnb.predict(X_test_prepared)
print(f1_score(quan_test,predictions,average='macro'))
fig,ax = plt.subplots(figsize=(20, 10))
cm = confusion_matrix(quan_test,predictions,labels=np.unique(newquantity))
disp = ConfusionMatrixDisplay(cm, display_labels=np.unique(newquantity))
disp.plot(ax=ax)
plt.show()
print(gnb.score(X_test_prepared, quan_test))

In [39]:
gnb = RandomForestClassifier(n_estimators=500)
gnb.fit(X_train_prepared,quan_train)
predictions = gnb.predict(X_test_prepared)
print(f1_score(quan_test,predictions,average='macro'))
fig,ax = plt.subplots(figsize=(20, 10))
cm = confusion_matrix(quan_test,predictions,labels=np.unique(newquantity))
disp = ConfusionMatrixDisplay(cm, display_labels=np.unique(newquantity))
disp.plot(ax=ax)
plt.show()
print(gnb.score(X_test_prepared, quan_test))

In [40]:
gnb = MLPClassifier(solver='sgd',max_iter=2000)
gnb.fit(X_train_prepared,quan_train)
predictions = gnb.predict(X_test_prepared)
print(f1_score(quan_test,predictions,average='macro'))
fig,ax = plt.subplots(figsize=(20, 10))
cm = confusion_matrix(quan_test,predictions,labels=np.unique(newquantity))
disp = ConfusionMatrixDisplay(cm, display_labels=np.unique(newquantity))
disp.plot(ax=ax)
plt.show()
print(gnb.score(X_test_prepared, quan_test))

In [41]:
gnb = MLPClassifier(solver='sgd',max_iter=2000,activation="logistic")
gnb.fit(X_train_prepared,quan_train)
predictions = gnb.predict(X_test_prepared)
print(f1_score(quan_test,predictions,average='macro'))
fig,ax = plt.subplots(figsize=(20, 10))
cm = confusion_matrix(quan_test,predictions,labels=np.unique(newquantity))
disp = ConfusionMatrixDisplay(cm, display_labels=np.unique(newquantity))
disp.plot(ax=ax)
plt.show()
print(gnb.score(X_test_prepared, quan_test))

In [42]:
def zip_directory(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, mode='w') as zipf:
        len_dir_path = len(folder_path)
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, file_path[len_dir_path:])
                os.remove(file_path)
                
zip_directory("/kaggle/working/", 'pleasework.zGR
ip')

In [ ]:
!ls

In [ ]:
zip_directory("/kaggle/working/", 'remaining.zip')

In [ ]:
os.remove("remaining.zip")

In [ ]:
from IPython.display import FileLink
FileLink(r'remaining.zip')

In [ ]:
import bz2
data = bz2.BZ2File("/kaggle/working/ImagesPixels319301.pbz2", 'rb')
data = cPickle.load(data)
print(data.shape)

In [ ]:
import pickle
import bz2
import _pickle as cPickle
for i,image in enumerate(imagefiles):
    #open_file = open("ImagesPixels"+str(i)+".pkl", "wb")
    with bz2.BZ2File("ImagesPixels"+str(i)+".pbz2", 'w') as f: 
        cPickle.dump(image, f)
    #pickle.dump(image, open_file)
    #open_file.close()
    #np.savetxt("ImagesPixels"+str(i+1)+".csv", image, delimiter=',')

In [ ]:
from PIL import Image
import pickle
import bz2
import _pickle as cPickle
imagefiles = []
jj = 0
for i,img in enumerate(newimagedata):
    if i%25==0:
        if jj in keepos:
            img1 = Image.open(img).convert("RGB")
            img1 = img1.resize((180,180),Image.BILINEAR)
            #150,150
            img1.save("Image1.jpg")
            #imagefiles.append(imread("Image1.jpg",as_gray = True))
            image = imread("Image1.jpg")
            #edges_prewitt_vertical = prewitt_v(image)
            imagefiles.append(image)
        jj = jj+1
        #with bz2.BZ2File("ImagesPixels"+str(i+1)+".pbz2", 'w') as f: 
        #    cPickle.dump(image, f)